In [ ]:
# Yonghwan Kim Assignment3 Task2 simple search engine-2: TF_query
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 41.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9f889d6dffccdbd2cc17d6dda478281ee2dd55ed90d025dbcf89fa953acddfdc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import sys
import re
import math
from pyspark import SparkContext, SparkConf

In [ ]:
# create Spark context with necessary configuration
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('Task2query')
sc = SparkContext(conf=conf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Give **.txt FILE PATHS** here
# You can use any random input file of your choice
TF_indexDir = '/content/drive/MyDrive/Assignment3_cloud/TF_index/part-00000'
stopwordDir = '/content/drive/MyDrive/Assignment3_cloud/stopwords-en.txt'
fileSaveDir = '/content/drive/MyDrive/Assignment3_cloud/Task2Output'
# USE THIS FILE as input FOR ALL BELOW tasks
# Change filepath to HDFS path when executing in the cluster

In [ ]:
# recover the string to RDD
# take input file from TF_index directory
fileRDD = sc.textFile(TF_indexDir)
# Split by word
recoverToRDD1 = fileRDD.map(lambda i: i.split('@'))
# split by doc id
recoverToRDD2 = recoverToRDD1.map(lambda i: (i[0], i[1].split('+')))
# split doc id and term frequency
recoverToRDD3 = recoverToRDD2.map(lambda i: (i[0], [x.split('#') for x in i[1]]))

In [ ]:
print('Enter your query: ')

# example string:
# sculpture marker coords dms nearest concolor

# stopword broadcasting
broadcastStopword = sc.broadcast(list(open(stopwordDir).read().splitlines()))
# take input string and change it to RDD
queryStrRDD = sc.parallelize(input().split(" "))
# case folding
casefoldingRDD = queryStrRDD.map(lambda line: line.lower())
# remove urls
URL_Kill = re.compile(r'(https?:|www.)\S+')
URLRemoverRDD = casefoldingRDD.map(lambda word: re.sub(URL_Kill, " ", word))
# remove all non-alphabetic chracters and words
punctuationsRemovalRDD = URLRemoverRDD.flatMap(lambda word: re.sub(r'[^a-zA-Z ]', ' ', word).split(" "))
# remove empty elements
emptyRemoveRDD = punctuationsRemovalRDD.filter(lambda word: word != '')
# remove stopwords
stopwordsRemoval = emptyRemoveRDD.filter(lambda word: word not in broadcastStopword.value)
# broadcast the input query
broadcastQuery = sc.broadcast(stopwordsRemoval.collect())

Enter your query: 
sculpture marker coords dms nearest concolor


In [ ]:
# filter the RDD with broadcasted values
filterQueryRDD = recoverToRDD3.filter(lambda word: word[0] in broadcastQuery.value)
# transform RDD to drop words
dropwordsRDD = filterQueryRDD.flatMap(lambda q: q[1])
# make the freq value float type to reduce later
floatTFRDD = dropwordsRDD.map(lambda i: (i[0], float(i[1])))
# combine all freq values
reduceQueryRDD = floatTFRDD.reduceByKey(lambda a, b: a + b)
# sort rdd in decreasing order
sortRDD = reduceQueryRDD.sortBy(lambda x: x[1], ascending=False)
# take top 10 results
finalRDD = sc.parallelize(sortRDD.take(10))
# save the text file to Task2Output directory
finalRDD.saveAsTextFile(fileSaveDir)